<a href="https://colab.research.google.com/github/rahulrajpr/AdvancedHyperParameterOptimisation/blob/Optuna/Optuna_MutipleModel_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Optuna Multi Model Search**

You can perform muti-model - multi hyperparameter seacched with optune inside ithe objective function with ease

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.datasets import load_breast_cancer

In [3]:
dataset = load_breast_cancer()
data = dataset.data
target = dataset.target
columns = dataset.feature_names
df = pd.DataFrame(data = data, columns = columns)
df['target'] = target
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

In [4]:
X = df.drop('target', axis =1)
y = df['target']

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train,X_test,y_train, y_test = train_test_split(X,y,train_size = 0.7, random_state = 100, stratify = y)

Define the objective function

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [11]:
from sklearn.model_selection import cross_val_score

In [12]:
model_list = ['Logit','RF','GBC']

In [30]:
def objective(trial):
    
    classifier_name = trial.suggest_categorical("classifier", ["logit", "RF", 'GBM'])
    
    if classifier_name == "logit":
        
        logit_penalty = trial.suggest_categorical('logit_penalty', ['l1','l2'])
        logit_c = trial.suggest_float('logit_c', 0.001, 10)
        logit_solver = 'saga'
        
        model = LogisticRegression(
            penalty=logit_penalty,
            C=logit_c,
            solver=logit_solver,
        )
        
    elif classifier_name =="RF":
        
        rf_n_estimators = trial.suggest_int("rf_n_estimators", 100, 1000)
        rf_criterion = trial.suggest_categorical("rf_criterion", ['gini', 'entropy'])
        rf_max_depth = trial.suggest_int("rf_max_depth", 1, 4)
        rf_min_samples_split = trial.suggest_float("rf_min_samples_split", 0.01, 1)

        model = RandomForestClassifier(
            n_estimators=rf_n_estimators,
            criterion=rf_criterion,
            max_depth=rf_max_depth,
            min_samples_split=rf_min_samples_split,
        )
        
    else:
        
        gbm_n_estimators = trial.suggest_int("gbm_n_estimators", 100, 1000)
        gbm_criterion = trial.suggest_categorical("gbm_criterion", ['mse', 'friedman_mse'])
        gbm_max_depth = trial.suggest_int("gbm_max_depth", 1, 4)
        gbm_min_samples_split = trial.suggest_float("gbm_min_samples_split", 0.01, 1)

        model = GradientBoostingClassifier(
            n_estimators=gbm_n_estimators,
            criterion=gbm_criterion,
            max_depth=gbm_max_depth,
            min_samples_split=gbm_min_samples_split,
        )

    
    score = cross_val_score(model, X_train, y_train, cv=3)
    accuracy = score.mean()
    
    return accuracy

**Random Search**

In [15]:
from IPython.core.display import clear_output
!pip install optuna
clear_output()

In [16]:
import optuna

In [31]:
study = optuna.create_study(direction = 'maximize',
                            sampler = optuna.samplers.TPESampler())

[I 2023-01-04 07:18:16,276] A new study created in memory with name: no-name-3dc455cf-761d-4751-9e58-66fc362f4018


In [32]:
study.optimize(func = objective,
               n_trials = 100,
               n_jobs = -1, 
               show_progress_bar = True)

%time

  0%|          | 0/100 [00:00<?, ?it/s]

[I 2023-01-04 07:18:19,830] Trial 0 finished with value: 0.9045150755677072 and parameters: {'classifier': 'logit', 'logit_penalty': 'l2', 'logit_c': 2.169858972762116}. Best is trial 0 with value: 0.9045150755677072.
[I 2023-01-04 07:18:19,838] Trial 1 finished with value: 0.9045150755677072 and parameters: {'classifier': 'logit', 'logit_penalty': 'l2', 'logit_c': 8.398176026600044}. Best is trial 0 with value: 0.9045150755677072.
[I 2023-01-04 07:18:20,042] Trial 3 finished with value: 0.9045150755677072 and parameters: {'classifier': 'logit', 'logit_penalty': 'l1', 'logit_c': 0.06717505827657361}. Best is trial 0 with value: 0.9045150755677072.
[I 2023-01-04 07:18:26,314] Trial 2 finished with value: 0.9371534897850687 and parameters: {'classifier': 'RF', 'rf_n_estimators': 839, 'rf_criterion': 'gini', 'rf_max_depth': 2, 'rf_min_samples_split': 0.012027421383948056}. Best is trial 2 with value: 0.9371534897850687.
[I 2023-01-04 07:18:28,434] Trial 4 finished with value: 0.9396977291

In [33]:
study.best_value

0.9647983595352017

In [36]:
best_params = study.best_params
best_params

{'classifier': 'GBM',
 'gbm_n_estimators': 557,
 'gbm_criterion': 'mse',
 'gbm_max_depth': 4,
 'gbm_min_samples_split': 0.7226020165103426}

In [41]:
best_classifier = best_params.pop('classifier')
best_classifier

'GBM'

In [51]:
best_params

{'gbm_n_estimators': 557,
 'gbm_criterion': 'mse',
 'gbm_max_depth': 4,
 'gbm_min_samples_split': 0.7226020165103426}

In [52]:
params_load = { ('_').join(x.split('_')[1:]):y for x,y in best_params.items()}
params_load

{'n_estimators': 557,
 'criterion': 'mse',
 'max_depth': 4,
 'min_samples_split': 0.7226020165103426}

In [53]:
gbm = GradientBoostingClassifier(**params_load)
gbm.fit(X_train,y_train)

GradientBoostingClassifier(criterion='mse', max_depth=4,
                           min_samples_split=0.7226020165103426,
                           n_estimators=557)

In [54]:
from sklearn.metrics import accuracy_score

In [55]:
def evaluate_model(model):
  print('train accuracy : ',accuracy_score(y_train,model.predict(X_train)))
  print('test accuracy : ',accuracy_score(y_test,model.predict(X_test)))

In [56]:
evaluate_model(gbm)

train accuracy :  1.0
test accuracy :  0.9649122807017544
